In [87]:
import h5py
import os
import zlib
import msgpack
import numpy as np
import scipy

def read_compressed_json(path):
	if not os.path.isfile(path):
		return {}
	with open(path, 'rb') as f:
		o = f.read()
		o = zlib.decompress(o)
		o = msgpack.unpackb(o, strict_map_key=False)
		return o

In [88]:
TRAINED_STUDIES = sorted(os.listdir('/home/ub-sonvo-25d094476064960/celltype_prediction/camel_2'))
GENES_ANNOTATION = read_compressed_json('/home/ub-sonvo-25d094476064960/genes_annotation.json.gz')
N_GENES = GENES_ANNOTATION['n_genes']

In [89]:
ROOT_DIR = '/home/ub-sonvo-25d094476064960/celltype_prediction/camel_2'
TERM_MAPPING = read_compressed_json('/home/ub-sonvo-25d094476064960/term_mapping.json.gz')

FIBROBLAST_IDX = TERM_MAPPING['name2idx']['fibroblast']
print(FIBROBLAST_IDX)

SUBCELLTYPE_IDX = TERM_MAPPING['name2idx']['cell type - subgroup']
print(SUBCELLTYPE_IDX)

CONNECTIVE_TISSUE_CELL_IDX = TERM_MAPPING['name2idx']['connective tissue cell']
print(CONNECTIVE_TISSUE_CELL_IDX)

387
226
278


In [91]:
merged_meta_arr = np.load(
    '/home/ub-sonvo-25d094476064960/celltype_prediction/ML_DL/trained_data_2/trained_meta_celltypes_HVG.npy'
)

In [92]:
merged_meta_arr = np.load(
    '/home/ub-sonvo-25d094476064960/celltype_prediction/ML_DL/trained_data_2/new_tree_major_HVG.npy'
)

In [94]:
studies_idx_arr = []
for i in range(len(TRAINED_STUDIES)):
    study_idx = TRAINED_STUDIES[i]
    hdf5_path = os.path.join(ROOT_DIR, str(study_idx), 'raw.hdf5')
    with h5py.File(hdf5_path) as f:
        bc = f['barcodes'][()]
    tmp = np.zeros(shape=len(bc))
    tmp[:] = i
    studies_idx_arr.append(tmp)

In [95]:
studies_idx_arr = np.concatenate(studies_idx_arr)
studies_idx_arr = studies_idx_arr.astype(np.uint16)
# np.save('trained_data_2/new_tree_studies_idx_arr.npy', studies_idx_arr)

In [76]:
studies_idx_arr = np.load(
    '/home/ub-sonvo-25d094476064960/celltype_prediction/ML_DL/trained_data_2/new_tree_studies_idx_arr.npy'
)
studies_idx_arr

array([  0,   0,   0, ..., 149, 149, 149])

In [78]:
np.save(
    '/home/ub-sonvo-25d094476064960/celltype_prediction/ML_DL/trained_data_2/new_tree_studies_idx_arr.npy',
    studies_idx_arr.astype(np.uint16)
)

In [98]:
merged_meta_arr

array([359, 359, 359, ...,   0,   0,   0])

In [99]:
celltype_idx = np.unique(merged_meta_arr)
chosen_genes_arr = np.ones(shape=(len(celltype_idx), N_GENES), dtype=np.bool_)

for i in range(len(TRAINED_STUDIES)):
    print(i)
    tmp_arr = np.zeros(shape=chosen_genes_arr.shape, dtype=np.bool_)
    tmp_arr[:] = True
    
    study_idx = TRAINED_STUDIES[i]
    tmp_cell_types_arr = merged_meta_arr[studies_idx_arr == i]
    for k in np.unique(tmp_cell_types_arr):
        tmp_arr[
            np.nonzero(celltype_idx == k)[0],
            :
        ] = False
        
    with h5py.File('{}/{}/{}'.format(ROOT_DIR, study_idx, 'raw.hdf5')) as f:
        indptr = f['expression']['indptr'][()]
        indices = f['expression']['indices'][()]
    
    tmp_indptr_counts = indptr[1:] - indptr[:-1]
    tmp_indptr_counts = tmp_indptr_counts[:N_GENES]
    for k in range(len(tmp_indptr_counts)):
        if tmp_indptr_counts[k] == 0:
            continue
        tmp_indices = indices[indptr[k] : indptr[k+1]]
        tmp_indices = tmp_cell_types_arr[tmp_indices]
        
        for j in np.unique(tmp_indices):
            tmp_arr[
                np.nonzero(celltype_idx == j)[0],
                k
            ] = True

    # print (tmp_arr)
    # print (chosen_genes_arr)

    chosen_genes_arr = (chosen_genes_arr.astype(np.uint8) + tmp_arr.astype(np.uint8)) == 2

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


In [82]:
chosen_genes_arr.shape

(37, 63683)

In [100]:
np.sum(np.sum(chosen_genes_arr, axis=0) > 1)

28505

In [101]:
np.save(
    'trained_data_2/new_tree_genes_bool_2.npy',
     np.sum(chosen_genes_arr, axis=0) > 1   
)